In [1]:
import numpy as np
import numpy.linalg as la
import scipy.signal as scisig

from func import *

### Introduction

Previously we implemented 1D and 2D convolution using Winograd. We saw the potential $2 \times$ savings of it compared to the direct method. Here, we seek to generalize the size of the filter and ouput of the FIR tap $F(m,r)$ using the previous work of Toom-Cook.


### F(2,3)

We define the matrices below for our bilinear algorithm

$$Y = A^T \Big[(Gg) \odot (B^Td) \Big]$$

The algorithm seeks to solve

$$
\begin{bmatrix}
d_0 & d_1 & d_2 \\
d_1 & d_2 & d_3 
\end{bmatrix}
\begin{bmatrix}
g_0 \\ g_1 \\ g_2 
\end{bmatrix}
$$

Here, the input size is $\alpha = m+r-1$, the filter size is $r$, and the output size is $r$. Winograd showed we only need to compute $\alpha$ multiplications, which is equivalent to the $\alpha \times \alpha$ matrix multiplication.

In [2]:
B23 = np.asarray([
    [1, 0,-1, 0],
    [0, 1, 1, 0],
    [0,-1, 1, 0],
    [0, 1, 0,-1]
]).T

G23 = np.asarray([
    [ 1,  0, 0],
    [.5, .5,.5],
    [.5,-.5,.5],
    [ 0, 0,  1]
])

A23 = np.asarray([
    [1,1,1,0],
    [0,1,-1,-1]
]).T

In [3]:
d = np.random.random(4)
g = np.random.random(3)

inner = np.dot(G23,g) * np.dot(B23.T,d)
y = np.dot(A23.T, inner)

direct = np.asarray([sum(d[:3]*g), sum(d[1:]*g)])

print("Error:",la.norm(direct - y)/la.norm(direct))

Error: 1.7575913666155722e-16


For this variation, we see the algorithm requires $(24,28)$ flops.

### 2D Case 

The Winograd extended to 2D is

$$Y = A^T \Big[(GgG^T) \odot (B^TdB) \Big]A$$

The extension to the Hankel case is we need a Doubly Toeplitz Matrix:

$$
\begin{bmatrix}
H_1 & H_3 & H_2\\
H_2 & H_1 & H3 \\
\end{bmatrix}
\begin{bmatrix}
X_3^T \\ X_2^T \\ X_1^T
\end{bmatrix}
$$

For each entry of the 2D convolved output, it relies on the sum of 1D convolutions. As we move across the rows and columnrs of the output, we convolve as well by selecting different submatrices of the data and filter. The doubly Toeplitz captures this idea. We also need to add a $0$ padding since we convolve "off" the data.

For more info, here are the [slides](http://www.cs.uoi.gr/~cnikou/Courses/Digital_Image_Processing/Chapter_04c_Frequency_Filtering_(Circulant_Matrices).pdf). We've implemented this code before and will use it.

In [4]:
g = np.random.random((4,4))
f = np.random.random((3,3))

direct = np.zeros((2,2))
for i in range(2):
    for j in range(2):
        direct[i,j] = np.sum(f * g[i:i+3,j:j+3])
        
inner = np.dot(G23, np.dot(f, G23.T)) * np.dot(B23.T, np.dot(g, B23))
Y = np.dot(A23.T, np.dot(inner, A23))

print("Error of one Winograd",la.norm(Y - direct)/la.norm(direct))

Error of one Winograd 1.655706559043613e-16


### Complete Winograd Convolution

In [5]:
convLib = scisig.convolve2d(f,g)
conv2d = convolve2DToeplitz(f,g)
g2 = revMatrix(g)
g2 = padImage(g2,len(f))
cWino = simpleWinogradAlg(f,g2,2,B23,G23,A23)[0]
cWino = revMatrix(cWino)

print("Error:",la.norm(convLib - conv2d, ord=2)/la.norm(convLib, ord=2))
print("Error:",la.norm(convLib - cWino, ord=2)/la.norm(convLib, ord=2))

Error: 8.103783236736365e-17
Error: 1.615305964141385e-16


### Hankel Representation

To restate the Winograd, we propose the Hankel multiplication

$$
\begin{bmatrix}
0 & 0 & 0 & d_0 \\
0 & 0 & d_0 & d_1 \\
0 & d_0 & d_1 & d_2 \\
d_0 & d_1 & d_2 & d_3 \\
d_1 & d_2 & d_3 & 0 \\
d_2 & d_3 & 0 & 0 \\
d_3 & 0 & 0 & 0
\end{bmatrix}
\begin{bmatrix}
0 \\ g_0 \\ g_1 \\ g_2 
\end{bmatrix}
$$

As a reduced convolution (only keep the top $n$ terms), this is

$$
\begin{bmatrix}
0 & 0 & 0 & d_0 \\
0 & 0 & d_0 & d_1 \\
0 & d_0 & d_1 & d_2 \\
d_0 & d_1 & d_2 & d_3 
\end{bmatrix}
\begin{bmatrix}
0 \\ g_0 \\ g_1 \\ g_2 
\end{bmatrix}
$$

For Winograd, this is further reduced to

$$
\begin{bmatrix}
0 & d_0 & d_1 & d_2 \\
d_0 & d_1 & d_2 & d_3 
\end{bmatrix}
\begin{bmatrix}
0 \\ g_0 \\ g_1 \\ g_2 
\end{bmatrix}
$$

In [6]:
d = np.random.random(4)
g = np.random.random(3)
g2 = np.append(np.zeros(1),g)
direct = np.dot(A23.T,  (np.dot(G23,g) * np.dot(B23.T,d))  )
print(direct)

H = toeplitzToHankle(vectorToToeplitz(d))[2:4]
conv = np.dot(H,g2)
print(conv)

[0.3434141  0.27206016]
[0.3434141  0.27206016]


### Toom-Cook Form

Let $\tilde{V}$ be the low-order evaluation matrix. Let $V^{-1}$ be the interpolation matrix. We recast this problem generally to

$$Y = V^{-1} \Big[(\tilde{V}g) \odot (\tilde{V}d) \Big]$$

This is equivalent to the Hankel form of:

$$
\begin{bmatrix}
0 & 0 & 0 & d_0 \\
0 & 0 & d_0 & d_1 \\
0 & d_0 & d_1 & d_2 \\
d_0 & d_1 & d_2 & d_3 
\end{bmatrix}
\begin{bmatrix}
0 \\ g_0 \\ g_1 \\ g_2 
\end{bmatrix}
$$

In [7]:
convTC = monomialTC(d,g2[::-1])

print(convTC[1],"\n")
print("2nd to 4th terms:",convTC[1][2:4])

[6.43787247e-01 4.79244490e-01 3.43414095e-01 2.72060160e-01
 5.36882881e-02 5.02780253e-02 3.70074342e-18] 

2nd to 4th terms: [0.3434141  0.27206016]


### Monomial Toom Cook Bilinear Algorithm

In [27]:
V = monoEval(4,False,True)

# multiplication
mult = np.dot(V,g2[::-1]) * np.dot(V,d)

# interpolation
V = monoInterp(4,False,True)
Vinv = la.inv(V)[2:4]
coeff = np.dot(Vinv,mult)

print(coeff)

[0.3434141  0.27206016]


### Chebyshev Toom-Cook Bilinear Algorithm

Below is the 1D case

$$
\begin{bmatrix}
0 & 0 & 0 & d_0 \\
0 & 0 & d_0 & d_1 \\
0 & d_0 & d_1 & d_2 \\
d_0 & d_1 & d_2 & d_3 
\end{bmatrix}
\begin{bmatrix}
g_0 \\ g_1 \\ g_2 \\ 0
\end{bmatrix}
$$

In [28]:
g3 = np.append(g, np.zeros(1))

n = len(d)
assert(n == len(g2))
    
z = 2*n-1
i = np.arange(n, dtype=np.float64)
j = np.arange(z, dtype=np.float64)

# Chebyshev nodes:
t = np.cos((2*j+1)/(2*z)*np.pi)

# dim: (nodes, i)
V = np.cos(i * np.arccos(t.reshape(-1, 1)))

P = np.dot(V,d)
Q = np.dot(V,g3[::-1])

# multiply
R = P*Q

# interpolate
V = np.cos(j * np.arccos(t.reshape(-1, 1)))
Vinv = la.inv(V)[:]*2
chebyConv = np.dot(Vinv, R)

print(chebyConv)

print("Minimal Filtering:",4)
print("Evaluation Shape:",V.shape)
print("Interpolation Shape:",Vinv.shape)

[0.38801489 1.4714925  0.89281228 0.55659847 0.27206016 0.05368829
 0.05027803]
Minimal Filtering: 4
Evaluation Shape: (7, 7)
Interpolation Shape: (7, 7)
